<a href="https://colab.research.google.com/github/deb-kit2/DeepDream/blob/master/DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Dependencies

In [1]:
from keras.applications import inception_v3
from keras import backend as k

k.set_learning_phase(0)

import numpy as np
import scipy
from keras.preprocessing.image import load_img, save_img, img_to_array

Using TensorFlow backend.


###hello


In [0]:
some_layers= {
    "mixed2" : 2.,
    "mixed3" : 0.5,
    "mixed4" : 1.,
    "mixed5" : 0.2, 
}

In [4]:
model = inception_v3.InceptionV3(include_top = False, weights = "imagenet")

loss = k.variable(0.)

for layer, coeff in some_layers.items():
  activation = model.get_layer(layer).output
  loss += coeff * k.sum(k.square(activation))

In [0]:
my_image = model.input

gradient = k.gradients(loss, my_image)
gradient /= k.maximum(k.mean(k.abs(gradient)), k.epsilon())

fetch = k.function([my_image], [loss, gradient])

def ascent(x, step, iterations, maxLoss = None):
  for i in range(iterations):
    l, g = fetch([x])
    if maxLoss is not None and l < maxLoss:
      break
    if(i%50 == 0):
      print("loss for " + str(i) + "th iteration : " + str(l))
    x += step*g
  return x

###Helpers

In [0]:
def resize(img, shaper):
  if k.img_data_format() == "channels_first":
    zoom = (1, 1, float(shaper[0]/img.shape[2]) , float(shaper[1]/img.shape[3]))
  else:
    zoom = (1, float(shaper[0]/img.shape[1]) , float(shaper[1]/img.shape[2]), 1)
  
  return scipy.ndimage.zoom(img, zoom, order = 1)

In [0]:
def preprocess(path):
  img = load_img(path)
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = inception_v3.preprocess_imput(img)
  return img

def deprocess(img):
  if k.img_data_format() = "channels_first":
    img = img.reshape((3, img.shape[2], img.shape[3]))
    img = img.transpose((1, 2, 0))
  else:
    img = img.reshape((img.shape[1], img.shape[2], 3))
  
  img /= 2
  img += 0.5
  img *= 255
  img = np.clip(img, 0., 255.).as_type('uint8')
  return img